In [1]:
import math
import numpy as np
import pandas as pd

from collections import OrderedDict
from ipynb.fs.full.config import Config
from ipynb.fs.full.genco import CPGenco
from ipynb.fs.full.auctioneer import clearing_mechanism, plot

import sys  
# sys.path.insert(1, '/home/sanjay/Research/MCTS/Codes/pda_simulator/notebooks/brokers')
sys.path.insert(1, '/mnt/d/PowerTAC/TCS/mcts_thread/pda_simulator/notebooks/brokers')
from ipynb.fs.full.zi import ZI
from ipynb.fs.full.zip import ZIP

import warnings
warnings.filterwarnings('ignore')

In [2]:
name_of_sellers = ['cp_genco']
name_of_buyers = ['buyer1', 'buyer2', 'buyer3']

list_of_sellers = dict()
list_of_buyers = dict()

config = Config()

for seller in name_of_sellers:
    seller_obj = CPGenco("cp_genco")
    list_of_sellers.update({seller: seller_obj})
    
buyer1 = ZI(config.market_demand*0.3, 5, id=name_of_buyers[0])
buyer2 = ZI(config.market_demand*0.3, 5, id=name_of_buyers[1])
buyer3 = ZIP(config.market_demand*0.4, 5, id=name_of_buyers[2])

list_of_buyers.update({name_of_buyers[0]: buyer1})
list_of_buyers.update({name_of_buyers[1]: buyer2})
list_of_buyers.update({name_of_buyers[2]: buyer3})

In [3]:
# PDA simulator

rounds = 24
cur_round = 0

while(cur_round < rounds):
    
    proximity = rounds - cur_round    
    
    # asks dataframe
    asks_df = pd.DataFrame(list_of_sellers['cp_genco'].asks(), columns=['ID', 'Price', 'Quantity'])

    # bids dataframe
    if proximity == 24:
        bids_df = pd.DataFrame([["miso", -1e9, np.random.normal(800, 100)]], columns=['ID', 'Price', 'Quantity'])
    else:
        bids_df = pd.DataFrame(columns=['ID', 'Price', 'Quantity'])
        
    for buyer in list_of_buyers.keys():
        buyer_df = pd.DataFrame(list_of_buyers[buyer].bids(rounds, cur_round+1), columns=['ID', 'Price', 'Quantity'])
        bids_df = pd.concat([bids_df,buyer_df], ignore_index=True)

    bids_df = bids_df.sort_values(by=['Price'])
                
    # market clearing
    mcp, mcq, cleared_asks_df, cleared_bids_df = clearing_mechanism(asks_df, bids_df)
    
    # update the cleared quantity of sellers
    for seller in list_of_sellers.keys():
        temp = cleared_asks_df.groupby('ID')
        if seller in temp.groups.keys():
            seller_cq = temp.sum()['Quantity'][seller]
            list_of_sellers[seller].set_cleared_quantity(-seller_cq)
        
    # update the cleared quantity of buyers
    for buyer in list_of_buyers.keys():
        temp = cleared_bids_df.groupby('ID')
        if buyer in temp.groups.keys():
            buyer_cq = temp.sum()['Quantity'][buyer]
            list_of_buyers[buyer].set_cleared_demand(buyer_cq)
            list_of_buyers[buyer].set_last_mcp(mcp)
    
    print('\n----------At Proxomity ', proximity, '------\n')
    print('MCP', mcp)
    print('MCQ', mcq)
    
    cur_round += 1


----------At Proxomity  24 ------

MCP 38.59251602381924
MCQ 746.8219159715836

----------At Proxomity  23 ------

MCP 44.26525420483663
MCQ 10.712633864224728

----------At Proxomity  22 ------

MCP 43.92091151856131
MCQ 45.70723782069217

----------At Proxomity  21 ------

MCP 54.54644588405971
MCQ 0.6427580318534837

----------At Proxomity  20 ------

MCP 48.95559365731288
MCQ 0.5784822286681361

----------At Proxomity  19 ------

MCP 45.270988942412785
MCQ 0.23139289146725447

----------At Proxomity  18 ------

MCP 61.586856474208965
MCQ 0.03085238552896783

----------At Proxomity  17 ------

MCP 49.04672967627518
MCQ 0.09872763369269819

----------At Proxomity  16 ------

MCP 53.833660971401144
MCQ 0.05

----------At Proxomity  15 ------

MCP 40.0
MCQ 0.0

----------At Proxomity  14 ------

MCP 40.0
MCQ 0.0

----------At Proxomity  13 ------

MCP 40.0
MCQ 0.0

----------At Proxomity  12 ------

MCP 40.0
MCQ 0.0

----------At Proxomity  11 ------

MCP 40.0
MCQ 0.0

----------At Pr